In [1]:
import numpy as np
import pyvista as pv

In [2]:
import torch

In [3]:
DXsec_Al = np.load('./DXsec_Al_1e8.npy')

In [4]:
import Multi_Species_etching_depo

In [5]:
def slide2D_fractionZ(film, start, end, direction, fraction, value):
    if fraction == '+':
        if direction == 'y':
            slit = np.linspace(0, np.abs(end[2] - start[2]), np.abs(end[1] - start[1]))
            fraction = np.abs(int(slit[0]-slit[1]))
            print('y', slit)
            print('fraction', fraction)
            for i in range(np.abs(end[1] - start[1])):
                if end[1] > start[1]:
                    film[start[0]:end[0], start[1] + i, start[2]:start[2] + int(slit[i])] = value
                    for j in range(fraction):
                        film[start[0]:end[0], start[1] + i,start[2]+int(slit[i])+j] = 1/(fraction+1)*(fraction-j)
                elif end[1] < start[1]:
                    film[start[0]:end[0], start[1] - i, start[2]:start[2] + int(slit[i])] = value
                    for j in range(fraction):
                        film[start[0]:end[0], start[1] - i,start[2]+int(slit[i])+j] = 1/(fraction+1)*(fraction-j)
        elif direction == 'x':
            slit = np.linspace(0, np.abs(end[2] - start[2]), np.abs(end[0] - start[0]))
            fraction = np.abs(int(slit[0]-slit[1]))
            print('x', slit)
            print('fraction', fraction)
            for i in range(np.abs(end[2] - start[2])):
                if end[0] > start[0]:
                    film[start[0] + i, start[1]:end[1], start[2]:start[2] + int(slit[i])] = value
                    for j in range(fraction):
                        film[start[0] + i, start[1]:end[1], start[2]+int(slit[i])+j] = 1/(fraction+1)*(fraction-j)
                elif end[0] < start[0]:
                    film[start[0] - i, start[1]:end[1], start[2]:start[2] + int(slit[i])] = value
                    for j in range(fraction):
                        film[start[0] - i, start[1]:end[1], start[2]+int(slit[i])+j] = 1/(fraction+1)*(fraction-j)
    elif fraction == '-':
        if direction == 'y':
            slit = np.linspace(0, np.abs(end[2] - start[2]), np.abs(end[1] - start[1]))
            fraction = np.abs(int(slit[0]-slit[1]))
            print('y', slit)
            print('fraction', fraction)
            for i in range(np.abs(end[1] - start[1])):
                if end[1] > start[1]:
                    film[start[0]:end[0], start[1] + i, start[2] - int(slit[i]):start[2]+1] = value
                    for j in range(fraction):
                        film[start[0]:end[0], start[1] + i,start[2]-int(slit[i])-j] = 1/(fraction+1)*(fraction-j)
                elif end[1] < start[1]:
                    film[start[0]:end[0], start[1] - i, start[2] - int(slit[i]):start[2]+1] = value
                    for j in range(fraction):
                        film[start[0]:end[0], start[1] - i,start[2]-int(slit[i])-j] = 1/(fraction+1)*(fraction-j)
        elif direction == 'x':
            slit = np.linspace(0, np.abs(end[2] - start[2]), np.abs(end[0] - start[0]))
            fraction = np.abs(int(slit[0]-slit[1]))
            print('x', slit)
            print('fraction', fraction)
            for i in range(np.abs(end[2] - start[2])):
                if end[0] > start[0]:
                    film[start[0] + i, start[1]:end[1], start[2] - int(slit[i]):start[2]+1] = value
                    for j in range(fraction):
                        film[start[0] + i, start[1]:end[1], start[2] - int(slit[i]):start[2]] = 1/(fraction+1)*(fraction-j)
                elif end[0] < start[0]:
                    film[start[0] - i, start[1]:end[1], start[2] - int(slit[i]):start[2]+1] = value
                    for j in range(fraction):
                        film[start[0] - i, start[1]:end[1], start[2] - int(slit[i]):start[2]] = 1/(fraction+1)*(fraction-j)
    return film

In [6]:
film = np.zeros((50, 200, 80))
fullcell = 60
bottom = 10

# film = slide2D_fractionZ(film=film, start=[0, 129, 30], end=[50, 120, 10], direction='y', fraction='-', value=10)
# film = slide2D_fractionZ(film=film, start=[0, 71, 30], end=[50, 80, 10], direction='y', fraction='-', value=10)

film[:, 80:121, 0:31] = fullcell

film[:, :, 0:bottom] = fullcell # bottom

In [1]:
film = np.zeros((50, 200, 180))

bottom = 10

film = slide2D_fractionZ(film=film, start=[0, 129, 30], end=[50, 120, 10], direction='y', fraction='-', value=10)
film = slide2D_fractionZ(film=film, start=[0, 71, 30], end=[50, 80, 10], direction='y', fraction='-', value=10)

film[:, 80:121, 0:31] = 10

film[:, :, 0:bottom] = 10 # bottom

NameError: name 'np' is not defined

In [7]:
np.arctan(9/20)/np.pi*180

24.22774531795417

In [8]:
depo1 = torch.Tensor(np.logical_and(film[:, :, :] !=0, film[:, :, :] < 10)).to_sparse()
depo1 = depo1.indices().numpy().T

substrute = torch.Tensor(film[:, :, :]==fullcell).to_sparse()
substrute = substrute.indices().numpy().T
depomesh = pv.PolyData(depo1)
depomesh["radius"] = np.ones(depo1.shape[0])*0.5
geom = pv.Box()

submesh = pv.PolyData(substrute)
submesh["radius"] = np.ones(substrute.shape[0])*0.5

# Progress bar is a new feature on master branch
depoglyphed = depomesh.glyph(scale="radius", geom=geom) # progress_bar=True)
subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)

p = pv.Plotter()
# p.add_mesh(depoglyphed, color='cyan')
p.add_mesh(subglyphed, color='dimgray')
p.enable_eye_dome_lighting()
p.show_grid()
p.show()

c:\Users\urcs049\AppData\Local\anaconda3\Lib\site-packages\pyvista\core\utilities\points.py:55: UserWarning: Points is not a float type. This can cause issues when transforming or applying filters. Casting to ``np.float32``. Disable this by passing ``force_float=False``.
  warnings.warn(
c:\Users\urcs049\AppData\Local\anaconda3\Lib\site-packages\pyvista\core\filters\data_set.py:2386: UserWarning: No vector-like data to use for orient. orient will be set to False.
  warnings.warn("No vector-like data to use for orient. orient will be set to False.")


Widget(value='<iframe src="http://localhost:59476/index.html?ui=P_0x12745c4b6b0_0&reconnect=auto" class="pyvis…

In [9]:
etchfilm = np.zeros((50, 200, 80, 2))
etchfilm[:, :, :, 0] = film

In [10]:
N = int(1e7)

In [11]:
logname = 'Multi_species_benchmark_0729'
testEtch = Multi_Species_etching_depo.etching(mirror=True,inputMethod='bunch', pressure_pa=0.001, temperature=300, chamberSize=etchfilm.shape,
                    depoThick=50, center_with_direction=np.array([[35,100,75]]), 
                    range3D=np.array([[0, 70, 0, 100, 0, 150]]), InOrOut=[1], yield_hist=np.array([None]),
                    reaction_type=False, param = [1.6, -0.7], N = N, 
                    sub_xy=[0,0], film=etchfilm, n=1, cellSize=etchfilm.shape, 
                    celllength=1e-5, kdtreeN=5, tstep=1e-5,
                    substrateTop=30,posGeneratorType='top',fullCell=fullcell, logname=logname)

In [12]:
deposit = testEtch.depo_position_increase_cosVel_normal(125, N, 2e-2, 10)

inputcount 5000


running: 100%|██████████████████████████████████████████████████████| 100/100 [14:28<00:00, 8.69s/B]


inputcount 5000


running: 100%|██████████████████████████████████████████████████████| 100/100 [20:46<00:00, 12.5s/B]


inputcount 5000


running:  43%|██████████████████████▊                              | 43.0/100 [12:08<16:05, 16.9s/B]

depo finish


In [ ]:
np.save('cvd_void_subDepo_thk110_tape115', deposit[0])

In [ ]:
deposit0 = np.load('./For_etching_transport_TS60_deposit_0604_4.npy')

In [14]:
deposit[0][20, 117, 30:]

array([[60.,  0.],
       [ 0., 60.],
       [ 0., 60.],
       [ 0., 60.],
       [ 0., 54.],
       [ 0.,  0.],
       [ 0.,  1.],
       [ 0., 15.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0., 32.],
       [ 0., 17.],
       [ 0.,  0.],
       [ 0., 20.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.]])

In [13]:
sumFilm = np.sum(deposit[0], axis=-1)

depo1 = torch.Tensor(np.logical_and(film[:, :, :,]!=fullcell, sumFilm[:, :, :,]!=0)).to_sparse()
depo1 = depo1.indices().numpy().T

substrute = torch.Tensor(film[:, :, :,]==fullcell).to_sparse()
substrute = substrute.indices().numpy().T
depomesh = pv.PolyData(depo1)
depomesh["radius"] = np.ones(depo1.shape[0])*0.5
geom = pv.Box()

submesh = pv.PolyData(substrute)
submesh["radius"] = np.ones(substrute.shape[0])*0.5

# Progress bar is a new feature on master branch
depoglyphed = depomesh.glyph(scale="radius", geom=geom) # progress_bar=True)
subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)

p = pv.Plotter()
p.add_mesh(depoglyphed, color='cyan')
p.add_mesh(subglyphed, color='dimgray')
p.enable_eye_dome_lighting()
p.show()

c:\Users\urcs049\AppData\Local\anaconda3\Lib\site-packages\pyvista\core\utilities\points.py:55: UserWarning: Points is not a float type. This can cause issues when transforming or applying filters. Casting to ``np.float32``. Disable this by passing ``force_float=False``.
  warnings.warn(
c:\Users\urcs049\AppData\Local\anaconda3\Lib\site-packages\pyvista\core\filters\data_set.py:2386: UserWarning: No vector-like data to use for orient. orient will be set to False.
  warnings.warn("No vector-like data to use for orient. orient will be set to False.")


Widget(value='<iframe src="http://localhost:59476/index.html?ui=P_0x1270d643e30_1&reconnect=auto" class="pyvis…

In [ ]:
deposit[2].shape

(19230, 6)

In [35]:
point_cloud = pv.PolyData(deposit[1][:, 3:])
vectors = deposit[1][:, :3]

point_cloud['vectors'] = vectors
arrows = point_cloud.glyph(
    orient='vectors',
    scale=1000,
    factor=2,
)

# Display the arrows
plotter = pv.Plotter()
plotter.add_mesh(point_cloud, color='maroon', point_size=5.0, render_points_as_spheres=True)
# plotter.add_mesh(arrows, color='lightblue')
# plotter.add_point_labels([point_cloud.center,], ['Center',],
#                          point_color='yellow', point_size=20)
plotter.show_grid()
plotter.show()

Widget(value='<iframe src="http://localhost:58622/index.html?ui=P_0x1f2bc0a65a0_4&reconnect=auto" class="pyvis…

In [36]:
point_cloud = pv.PolyData(deposit[2][:, 3:])
vectors = deposit[2][:, :3]

point_cloud['vectors'] = vectors
arrows = point_cloud.glyph(
    orient='vectors',
    scale=1000,
    factor=2,
)

# Display the arrows
plotter = pv.Plotter()
plotter.add_mesh(point_cloud, color='maroon', point_size=5.0, render_points_as_spheres=True)
# plotter.add_mesh(arrows, color='lightblue')
# plotter.add_point_labels([point_cloud.center,], ['Center',],
#                          point_color='yellow', point_size=20)
plotter.show_grid()
plotter.show()

Widget(value='<iframe src="http://localhost:58622/index.html?ui=P_0x1f2bbfb2330_5&reconnect=auto" class="pyvis…

In [21]:
deposit[0][20, 100, :]

array([[10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [ 0., 10.],
       [ 0.,  1.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,

In [ ]:
film = np.zeros((100, 100, 100, 2))

bottom = 10
film[:, :, 0:bottom, 0] = 10 # bottom

height = 80

film[:, :40, 0:height, 0] = 10
film[:, 60:, 0:height, 0] = 10
etchfilm = film


N = int(1e6)
velosity_matrix = np.zeros((N, 3))
tstep=1e-5
celllength=1e-5
# velosity_matrix[:, 0] = -1 * celllength /tstep
velosity_matrix[:, 1] = -1 * celllength /tstep
velosity_matrix[:, 2] = -1 * celllength /tstep

typeID = np.zeros(N)

print(velosity_matrix[0])

logname = 'Multi_species_benchmark_0729'
testEtch = Multi_Species_etching_depo.etching(mirror=True,inputMethod='bunch', pressure_pa=0.001, temperature=300, chamberSize=etchfilm.shape,
                    depoThick=90, center_with_direction=np.array([[35,100,75]]), 
                    range3D=np.array([[0, 70, 0, 100, 0, 150]]), InOrOut=[1], yield_hist=np.array([None]),
                    reaction_type=False, param = [1.6, -0.7], N = 300000, 
                    sub_xy=[0,0], film=etchfilm, n=1, cellSize=etchfilm.shape, 
                    celllength=1e-5, kdtreeN=5, tstep=1e-5,
                    substrateTop=40,posGeneratorType='benchmark',fullCell=10, logname=logname)


etching1 = testEtch.inputParticle(125, velosity_matrix, typeID, 2e-3, 20)

[ 0. -1. -1.]
inputcount 5000


running: 100%|██████████████████████████████████████████████████████| 100/100 [01:28<00:00, 1.13B/s]


In [ ]:
etching1[0][39, 60, :]

array([[10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,

In [ ]:
etching1[0][50, 60, :]

array([[10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,

In [ ]:
sumFilm = np.sum(etching1[0], axis=-1)

depo1 = torch.Tensor(np.logical_and(sumFilm[:, :, :,]!=10, sumFilm[:, :, :,]!=0)).to_sparse()
depo1 = depo1.indices().numpy().T

substrute = torch.Tensor(sumFilm[:, :, :,]==10).to_sparse()
substrute = substrute.indices().numpy().T
depomesh = pv.PolyData(depo1)
depomesh["radius"] = np.ones(depo1.shape[0])*0.5
geom = pv.Box()

submesh = pv.PolyData(substrute)
submesh["radius"] = np.ones(substrute.shape[0])*0.5

# Progress bar is a new feature on master branch
depoglyphed = depomesh.glyph(scale="radius", geom=geom) # progress_bar=True)
subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)

p = pv.Plotter()
p.add_mesh(depoglyphed, color='cyan')
p.add_mesh(subglyphed, color='dimgray')
p.enable_eye_dome_lighting()
p.show()

c:\Users\urcs049\AppData\Local\anaconda3\Lib\site-packages\pyvista\core\utilities\points.py:55: UserWarning: Points is not a float type. This can cause issues when transforming or applying filters. Casting to ``np.float32``. Disable this by passing ``force_float=False``.
  warnings.warn(
c:\Users\urcs049\AppData\Local\anaconda3\Lib\site-packages\pyvista\core\filters\data_set.py:2386: UserWarning: No vector-like data to use for orient. orient will be set to False.
  warnings.warn("No vector-like data to use for orient. orient will be set to False.")


Widget(value='<iframe src="http://localhost:53896/index.html?ui=P_0x2075d16a810_6&reconnect=auto" class="pyvis…

In [ ]:
point_cloud = pv.PolyData(etching1[1][:, 3:])
vectors = etching1[1][:, :3]

point_cloud['vectors'] = vectors
arrows = point_cloud.glyph(
    orient='vectors',
    scale=1000,
    factor=2,
)

# Display the arrows
plotter = pv.Plotter()
plotter.add_mesh(point_cloud, color='maroon', point_size=5.0, render_points_as_spheres=True)
# plotter.add_mesh(arrows, color='lightblue')
# plotter.add_point_labels([point_cloud.center,], ['Center',],
#                          point_color='yellow', point_size=20)
plotter.show_grid()
plotter.show()

Widget(value='<iframe src="http://localhost:53896/index.html?ui=P_0x2075d1b80e0_5&reconnect=auto" class="pyvis…

In [37]:
film = np.zeros((100, 100, 100, 2))

bottom = 10
fullcell = 100
film[:, :, 0:bottom, 0] = fullcell # bottom

height = 80

film[:, :40, 0:height, 0] = fullcell
film[:, 60:, 0:height, 0] = fullcell
etchfilm = film


N = int(1e6)
velosity_matrix = np.zeros((N, 3))
tstep=1e-5
celllength=1e-5
# velosity_matrix[:, 0] = -1 * celllength /tstep
velosity_matrix[:, 1] = -1 * celllength /tstep
velosity_matrix[:, 2] = -1 * celllength /tstep

typeID = np.zeros(N)

print(velosity_matrix[0])

logname = 'Multi_species_benchmark_0729'
testEtch = Multi_Species_etching_depo.etching(mirror=True,inputMethod='bunch', pressure_pa=0.001, temperature=300, chamberSize=etchfilm.shape,
                    depoThick=90, center_with_direction=np.array([[35,100,75]]), 
                    range3D=np.array([[0, 70, 0, 100, 0, 150]]), InOrOut=[1], yield_hist=np.array([None]),
                    reaction_type=False, param = [1.6, -0.7], N = 300000, 
                    sub_xy=[0,0], film=etchfilm, n=1, cellSize=etchfilm.shape, 
                    celllength=1e-5, kdtreeN=5, tstep=1e-5,
                    substrateTop=40,posGeneratorType='benchmark',fullCell=fullcell, logname=logname)


etching1 = testEtch.inputParticle(125, velosity_matrix, typeID, 2e-3, 20)

sumFilm = np.sum(etching1[0], axis=-1)

[ 0. -1. -1.]
inputcount 5000


running:   0%|                                                            | 0.00/100 [00:00<?, ?B/s]

weights_max 1
weights_min 1
weights_max 2
weights_min 1
weights_max 2
weights_min 1
weights_max 2
weights_min 1
weights_max 2
weights_min 2


running:   1%|▌                                                    | 1.00/100 [00:00<01:21, 1.22B/s]

weights_max 1
weights_min 0
weights_max 2
weights_min 1
weights_max 2
weights_min 1
weights_max 2
weights_min 1
weights_max 3
weights_min 2
weights_max 1
weights_min 0
weights_max 2
weights_min 1
weights_max 2
weights_min 1
weights_max 2
weights_min 1
weights_max 3
weights_min 2


...

running:  98%|███████████████████████████████████████████████████▉ | 98.0/100 [03:19<00:04, 2.39s/B]

weights_max 1
weights_min 1
weights_max 2
weights_min 1
weights_max 2
weights_min 1
weights_max 2
weights_min 1
weights_max 2
weights_min 2
weights_max 1
weights_min 1
weights_max 2
weights_min 1
weights_max 2
weights_min 1
weights_max 2
weights_min 1
weights_max 2
weights_min 2


running:  99%|████████████████████████████████████████████████████▍| 99.0/100 [03:22<00:02, 2.39s/B]

weights_max 1
weights_min 0
weights_max 2
weights_min 1
weights_max 2
weights_min 1
weights_max 2
weights_min 1
weights_max 2
weights_min 2
weights_max 1
weights_min 1
weights_max 2
weights_min 1
weights_max 2
weights_min 1
weights_max 2
weights_min 1
weights_max 2
weights_min 2


running: 100%|██████████████████████████████████████████████████████| 100/100 [03:24<00:00, 2.05s/B]


In [39]:
depo1 = torch.Tensor(np.logical_and(sumFilm[:, :, :,]!=10, sumFilm[:, :, :,]!=0)).to_sparse()
depo1 = depo1.indices().numpy().T

substrute = torch.Tensor(sumFilm[:, :, :,]==10).to_sparse()
substrute = substrute.indices().numpy().T
depomesh = pv.PolyData(depo1)
depomesh["radius"] = np.ones(depo1.shape[0])*0.5
geom = pv.Box()

submesh = pv.PolyData(substrute)
submesh["radius"] = np.ones(substrute.shape[0])*0.5

# Progress bar is a new feature on master branch
depoglyphed = depomesh.glyph(scale="radius", geom=geom) # progress_bar=True)
subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)

p = pv.Plotter()
p.add_mesh(depoglyphed, color='cyan')
p.add_mesh(subglyphed, color='dimgray')
p.enable_eye_dome_lighting()
p.show()

c:\Users\urcs049\AppData\Local\anaconda3\Lib\site-packages\pyvista\core\utilities\points.py:55: UserWarning: Points is not a float type. This can cause issues when transforming or applying filters. Casting to ``np.float32``. Disable this by passing ``force_float=False``.
  warnings.warn(
c:\Users\urcs049\AppData\Local\anaconda3\Lib\site-packages\pyvista\core\filters\data_set.py:2386: UserWarning: No vector-like data to use for orient. orient will be set to False.
  warnings.warn("No vector-like data to use for orient. orient will be set to False.")


Widget(value='<iframe src="http://localhost:54519/index.html?ui=P_0x2856e5e0440_9&reconnect=auto" class="pyvis…

In [42]:
etching1[0][45, 60, :]

array([[100.,   0.],
       [100.,   0.],
       [100.,   0.],
       [100.,   0.],
       [100.,   0.],
       [100.,   0.],
       [100.,   0.],
       [100.,   0.],
       [100.,   0.],
       [100.,   0.],
       [100.,   0.],
       [100.,   0.],
       [100.,   0.],
       [100.,   0.],
       [100.,   0.],
       [100.,   0.],
       [100.,   0.],
       [100.,   0.],
       [100.,   0.],
       [100.,   0.],
       [100.,   0.],
       [100.,   0.],
       [100.,   0.],
       [100.,   0.],
       [100.,   0.],
       [100.,   0.],
       [100.,   0.],
       [100.,   0.],
       [100.,   0.],
       [100.,   0.],
       [100.,   0.],
       [100.,   0.],
       [100.,   0.],
       [100.,   0.],
       [100.,   0.],
       [100.,   0.],
       [100.,   0.],
       [100.,   0.],
       [100.,   0.],
       [100.,   0.],
       [100.,   0.],
       [100.,   0.],
       [100.,   0.],
       [100.,   0.],
       [100.,   0.],
       [100.,   0.],
       [100.,   0.],
       [100.,

In [38]:
    # depo1 = torch.Tensor(np.logical_and(sumFilm[:, :, :,]!=10, sumFilm[:, :, :,]!=0)).to_sparse()
    # depo1 = depo1.indices().numpy().T

    # substrute = torch.Tensor(sumFilm[:, :, :,]==10).to_sparse()
    # substrute = substrute.indices().numpy().T
    # depomesh = pv.PolyData(depo1)
    # depomesh["radius"] = np.ones(depo1.shape[0])*0.5
    # geom = pv.Box()

    # submesh = pv.PolyData(substrute)
    # submesh["radius"] = np.ones(substrute.shape[0])*0.5

    # # Progress bar is a new feature on master branch
    # depoglyphed = depomesh.glyph(scale="radius", geom=geom) # progress_bar=True)
    # subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)

    # p = pv.Plotter()
    # p.add_mesh(depoglyphed, color='cyan')
    # p.add_mesh(subglyphed, color='dimgray')
    # p.enable_eye_dome_lighting()
    # p.show()

point_cloud = pv.PolyData(etching1[2][:, 3:])
vectors = etching1[2][:, :3]

point_cloud['vectors'] = vectors
arrows = point_cloud.glyph(
orient='vectors',
scale=1000,
factor=2,
)

# Display the arrows
plotter = pv.Plotter()
plotter.add_mesh(point_cloud, color='maroon', point_size=5.0, render_points_as_spheres=True)
# plotter.add_mesh(arrows, color='lightblue')
# plotter.add_point_labels([point_cloud.center,], ['Center',],
#                          point_color='yellow', point_size=20)
plotter.show_grid()
plotter.show()

Widget(value='<iframe src="http://localhost:54519/index.html?ui=P_0x285427d2a80_8&reconnect=auto" class="pyvis…

In [ ]:
point_cloud = pv.PolyData(etching1[2][:, 3:])
vectors = etching1[2][:, :3]

point_cloud['vectors'] = vectors
arrows = point_cloud.glyph(
orient='vectors',
scale=1000,
factor=2,
)

# Display the arrows
plotter = pv.Plotter()
plotter.add_mesh(point_cloud, color='maroon', point_size=5.0, render_points_as_spheres=True)
# plotter.add_mesh(arrows, color='lightblue')
# plotter.add_point_labels([point_cloud.center,], ['Center',],
#                          point_color='yellow', point_size=20)
plotter.show_grid()
plotter.show()